# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
# !pip3 install ratsnlp

# 구글 드라이브 연동하기

이번 튜토리얼에서 구축할 어휘 집합을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)

## 말뭉치 다운로드 및 전처리

오픈소스 파이썬 패키지 코포라(Korpora)를 활용해 BPE 수행 대상 말뭉치를 내려받고 전처리합니다. 실습용 말뭉치는 박은정 님이 공개하신 Naver Sentiment Movie Corpus(NSMC)입니다.

다음을 수행해 데이터를 내려받아 `nsmc`라는 변수로 읽어들입니다.

In [1]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 19.4MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 15.5MB/s]                            


다음을 수행하면 NSMC에 포함된 영화 리뷰(순수 텍스트)들을 지정된 경로에 저장해 둡니다.

In [2]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f:
        for line in lines:
            f.write(f'{line}\n')

write_lines("./content/train.txt", nsmc.train.get_all_texts())
write_lines("./content/test.txt", nsmc.test.get_all_texts())

`train.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head train.txt

`test.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head test.txt

## GPT 토크나이저 구축

GPT 계열 모델이 사용하는 토크나이저는 Byte-level Byte Pair Encoding(BBPE)입니다. 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 둡니다.

In [ ]:
# import os
# os.makedirs("/gdrive/My Drive/nlpbook/bbpe", exist_ok=True)

다음을 수행하면 `nsmc` 데이터를 가지고 BBPE 어휘집합을 구축합니다. BBPE 어휘집합 구축에 시간이 걸리니 잠시 기다려주세요. 

In [5]:
from tokenizers import ByteLevelBPETokenizer
bytebpe_tokenizer = ByteLevelBPETokenizer()
bytebpe_tokenizer.train(
    files=["./content/train.txt", "./content/test.txt"],
    vocab_size=10000,
    special_tokens=["[PAD]"]
)
bytebpe_tokenizer.save_model("./save")

['./save\\vocab.json', './save\\merges.txt']

위의 코드 수행이 끝나면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/bbpe`)에 `vocab.json`과 `merges.txt`가 생성됩니다. 전자는 바이트 레벨 BPE의 어휘 집합이며 후자는 바이그램 쌍의 병합 우선순위입니다. 

`vocab.json`은 다음과 같이 생겼습니다.

In [ ]:
!cat /gdrive/My\ Drive/nlpbook/bbpe/vocab.json

`merges.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /gdrive/My\ Drive/nlpbook/bbpe/merges.txt

## BERT 토크나이저 구축

BERT는 워드피스(wordpiece) 토크나이저를 사용합니다. 우선 어휘집합 구축 결과를 저장해둘 디렉토리를 자신의 구글 드라이브 계정 내 `내 드라이브/nlpbook/bbpe`로 만들어 둡니다.


In [ ]:
# import os
# os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok=True)

다음을 수행하면 BERT 모델이 사용하는 워드피스 어휘집합을 구축할 수 있습니다. 워드피스 어휘집합 구축에 시간이 걸리니 잠시만 기다려주세요.

In [6]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["./content/train.txt", "./content/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("./save")

['./save\\vocab.txt']

위의 코드 수행이 끝나면 자신의 구글 드라이브 경로(`/gdrive/My Drive/nlpbook/wordpiece`)에 `vocab.txt`가 생성됩니다. `vocab.txt`의 앞부분은 다음과 같이 생겼습니다.

In [ ]:
!head /gdrive/My\ Drive/nlpbook/wordpiece/vocab.txt